# CUDA OVERVIEW
[CUDA](https://developer.nvidia.com/cuda-toolkit)® is a parallel computing platform and programming model developed by [NVIDIA](https://www.nvidia.com) for general computing on graphical processing units ([GPUs](https://developer.nvidia.com/cuda-gpus)). With CUDA, developers are able to dramatically speed up computing applications by harnessing the power of GPUs.

In GPU-accelerated applications, the sequential part of the workload runs on the CPU – which is optimized for single-threaded performance – while the compute intensive portion of the application runs on thousands of GPU cores in parallel. When using CUDA, developers program in popular languages such as C, C++, Fortran, Python and MATLAB and express parallelism through extensions in the form of a few basic keywords.

The CUDA Toolkit from NVIDIA provides everything you need to develop GPU-accelerated applications. The CUDA Toolkit includes GPU-accelerated libraries, a compiler, development tools and the CUDA runtime.

The following notebook is only meant to expose the reader to the various ways to accelerate an application using GPUs. For more in depth training, please visit the [NVIDIA Developer Blog](https://devblogs.nvidia.com/) and  [NVIDIA Deep Learning Institue](https://www.nvidia.com/en-us/deep-learning-ai/education/). There you can access a wealth of resources on Deep Learning and Accelerated Computing Courses. And don't forget to signup for a [NVIDIA Developer Account](https://developer.nvidia.com/developer-program) to get access to all the latest information and toolsets!!!

In order to fully utilize this notebook, please follow the instructions at [README.md](https://github.com/mnicely/computeWorks_examples/tree/master/computeWorks_mm).

## Matrix Multiplication Tutorial

![mm](https://www.mathsisfun.com/algebra/images/matrix-multiply-order.gif)

Matrix multiplication is a great to see the power of parallel processing using a GPU. Matrix multiplication is what we like to call _embarrassingly parallel_, which simply means it takes little-to-no effort to separate individual tasks. 

An introductory video is provided in the next cell. Here is another nice tutorial [www.mathsisfun.com](https://www.mathsisfun.com/algebra/matrix-multiplying.html)

In [ ]:
%%HTML

<div align="center"><iframe src="https://www.youtube.com/embed/kqWCwwyeE6k?start=70" frameborder="0" width="900" height="550" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe></div>

## Hardware Check

First, lets confirm we have access to our GPU. We can do this using [nvidia-smi](https://developer.nvidia.com/nvidia-system-management-interface).

The NVIDIA System Management Interface (nvidia-smi) is a command line utility, based on top of the NVIDIA Management Library (NVML), intended to aid in the management and monitoring of NVIDIA GPU devices. 

This utility allows administrators to query GPU device state and with the appropriate privileges, permits administrators to modify GPU device state.  It is targeted at the TeslaTM, GRIDTM, QuadroTM and Titan X product, though limited support is also available on other NVIDIA GPUs.

To run a block of code below, click on it to select it and then you can either click the run (button with triangle) button in the menu above or type Ctrl+Enter:

In [ ]:
!nvidia-smi

## Software Check

Second, lets confirm we have access to our software stack.

We will be using the CUDA compiler driver, [NVCC](https://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html), and the [PGI Compiler](https://www.pgroup.com/products/community.htm).

It is the purpose of nvcc, the CUDA compiler driver, to hide the intricate details of CUDA compilation from developers. It accepts a range of conventional compiler options, such as for defining macros and include/library paths, and for steering the compilation process.

We will be using the PGI tool set for OpenMP and OpenACC. Note, it is possible to do this entire with to PGI compiler, but we'll use both for tutorial purposes.

In [ ]:
!nvcc -V

In [ ]:
!pgc++ -V

## Normal (Serial - CPU)
Lets begin this tutorial by baselining the serial version of general matrix multiplication, [GEMM](https://spatial-lang.org/gemm). Start by studying [normal_C.cpp](/edit/normal_C.cpp) containing the code. You will notice in the *normalC*() function that there is a set of nested for-loops. This is a telltale sign for areas of your code that have the **potential** to be distributed to a GPU.

Implement the following suggestions to better understanding how they can affect the compile and/or runtime performance.

1. Change _N_ to something smaller and larger than 1024. Keep in mind that the algorithm computational complexity is $(n^3)$, <em>N</em> > 1024 can take many seconds to complete.
2. Change the optimization flag between `-O0`, `-O1`, `-O2`, and `-O3` to see how compiler optimization effect the execution time.

In [ ]:
!nvcc -O2 normal_C.cpp -o normal_C

In [ ]:
!./normal_C 1024

## OpenMP
Now, lets jump into [compiler directives](https://en.wikipedia.org/wiki/Directive_(programming)). They are one of the easiest forms 0f optimization techniques and probably the most common for a CPU is [OpenMP](https://www.openmp.org/resources/). Using directives you are giving the compiler hints at compile time where you think further optimizations can be made. Start by studying [openmp.cpp](/edit/openmp.cpp) containing the code. Notice at the **#pragma** statement at line 73. The pragma is a directive telling the compiler *look here*.

In this example, we are giving the compiler the following hint:
`#pragma omp parallel for shared(A, B, C, n) private(i, j, k) schedule(static)`

We are telling the compiler which variables are shared along all the threads and which variables are private among each thread. Schedule tells the compiler the ordering method to execute threads.

With the current code, we need to set the system environment variable OMP_NUM_THREADS, with `export OMP_NUM_THREADS=X`, where X is the number of CPU threads available to use. Notice that in a Jupyter notebook we use the **%** magic command and `env`.

This system environment variable can be overwritten at runtime by using *set_omp_num_threads*(X), which can be found in the *omp.h* header file.

Lastly, we need to pass some additional flags to the nvcc compiler. In this tutorial, we are using the OpenMP library from the PGI compiler. Therefore, we need to clarify that we want to use the PGI C++ compiler for the host code. This is done with `-ccbin pgc++`. Now that we have chosen the compiler for the host code, we need to let the compiler know that we want to use PGI version 19.4 and link its OpenMP library using the `-mp` flag.

Implement the following suggestions to better understanding how they can affect the compile and/or runtime performance.

1. Change the number of threads available through the system environment variable.
2. Change the optimization flag between `-O0`, `-O1`, `-O2`, and `-O3`.
3. Uncomment line 25 and 66 to use OpenMP runtime API, then change the number of threads in `set_omp_num_threads(X)`.
4. Set the matrix to something smaller and larger than 1024.
5. Remove `-mp` from the build command.

In [ ]:
!nvcc -ccbin pgc++ -O2 -Xcompiler "-V19.4 -mp" openmp.cpp -o openmp

In [ ]:
%env OMP_NUM_THREADS=6

In [ ]:
!./openmp 1024

## OpenACC
As we just saw, utilizing parallel resources can greatly improve execution time of an application. Using OpenMP we were able to gain access to multiple cores on the CPU. But how do we access the hundreds, and even thousands, of cores on a GPU using compiler directives? That's where [OpenACC](https://developer.nvidia.com/openacc) comes in! This is great for legacy applications, where optimizations are wanted without an extensive overhaul of the code!

Just like OpenMP, OpenACC allows you create highly optimized code with little programming and low level knowledge of a GPU. Start by studying [openacc.cpp](/edit/openacc.cpp) containing the code. In the *openACC*() function, you will see **#pragma** statements just like we had in OpenMP. Using OpenACC we have finer granuality when assigning our code to the GPU. For that reason we need to give the compiler a few more hints. With these additional hints we tell the compiler where any dependences are in our code. The compiler will try it's best if these hints aren't there.

To use OpenACC, we need to pass the `-acc` flag to the compiler, just like when we passed `-mp` to use OpenMP. We can pass some more flags for increased functionality. First, we let the compiler know that we want to use a NVIDIA GPU with `-ta=tesla:nordc`. When we add `-Minfo=accel`, the compiler will display optimization analysis during compilcation. It will do this everytime it see `#pragma acc kernels`, which is a hint to compiler to analyze that region of code for optimizations. 

What if we want to profile the code for performance data? Your initial thought may be be to wrap the *openACC*() function with timers, like with did with the serial code. While this method will certainly give you general execution time, it leaves a lot of room for error and users are advised not to use this method. Instead, we can pass the `-ta=time` flag to the compiler without writing any extra code. Using this flag will also give us a breakdown of compute time and transfer times to and from the GPU. The flag `-Bstatic_pgi` links the required libraries for profiling. 

Notice that we don't need to set any system environment variables.

Please visit [NVIDIA DLI Education](https://www.nvidia.com/en-us/deep-learning-ai/education/) for an in-depth tutorial on OpenACC!

Implement the following suggestions to better understanding how they can affect the compile and/or runtime performance.

1. Change the optimization flag between `-O0`, `-O1`, `-O2`, and `-O3`.
2. Set the matrix to something smaller and larger than 1024.
3. Remove `-acc` from the build command.
4. Remove accel from the `-Minfo` flag in the build command.
5. Remove `-ta=time` from the build command

In [ ]:
!nvcc -ccbin pgc++ -O2 -Xcompiler "-V19.4 -acc -ta=tesla:nordc -Minfo=accel -Bstatic_pgi -ta=time" openacc.cpp -o openacc

From the compiler output above, we see the affect of adding the `-Minfo=accel` flag. It does a good explaining what's going on. First, it is generating the CUDA code required to transfer matrices A and B to GPU device memory. As well as the necessary code to transfer the resulting C matrix back to the host. Where it says *Generating Tesla code*, compute code is being generated to parallelize the for loops at lines 64, 66, and 69. 

In [ ]:
!./openacc 1024

By adding `-Bstatic_pgi -ta=time`, we get the kernel timing data broken down into various details. The total time to copy data to the device, run the computation, and copy the result back to the host memory takes 20 ms. Currently, we are only concerned in the time it take to do the actual matrix multiplication. For that we need to look at the section that says **XX: kernel launched X times**. Here we can see that the kernel was launched a total of 5 times. The next line tells us that a grid of 8 x 1024 blocks were launched with 128 threads per block. Then if we look at **device time(us)**, we can see that average kernel execution took rought 2 ms.

To learn more about the thread hiearchy, please read the [CUDA Programming Guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#programming-model).

## BLAS
An alternative to compiler directives is to use a drop-in [library](https://en.wikipedia.org/wiki/Library_(computing)). A library is a collection of precompiled routines that an application can use. They are often written by domain experts and optimized for particular hardware. This allows programming to focus on aspects of their application that is unique and/or custom.

The [Basic Linear Algerba Subroutines](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms) (BLAS) library is probably one of the most highly optimized library in the world. It also include a function specifically for GEMM calculations! Start by studying [blas.cpp](/edit/blas.cpp) containing the code. You will notice in the *blas*() function that there are no for loops. That is because they are *hidden* in the BLAS library. All we have to do is call the *blas_sgemm*() function and pass the appropiate parameters. 

Like OpenMP, BLAS can take advantage of muliple threads on a CPU, if resources are available. It does this using the same system environment variable OMP_NUM_THREADS, with `export OMP_NUM_THREADS=X`. Or the runtime API can be used through the *openblas_set_num_threads*(X) function, which can be found in the *cblas.h* header file.

In this example, we need to link the PGI BLAS library using `-Xlinker "-lblas"` in the build command.

Implement the following suggestions to better understanding how they can affect the compile and/or runtime performance.

1. Change the number of threads available through the system environment variable.
2. Change the optimization flag between `-O0`, `-O1`, `-O2`, and `-O3`.
3. Uncomment line 61 to use BLAS runtime API, then change the number of threads in `openblas_set_num_threads(X)`.
4. Set the matrix to something smaller and larger than 1024.

In [ ]:
!nvcc -ccbin pgc++ -O2 -Xlinker "-lblas" -Xcompiler "-V19.4" blas.cpp -o blas

In [ ]:
%env OMP_NUM_THREADS=4

In [ ]:
!./blas 1024

## cuBLAS
As we discovered in the previous section, drop-in libraries are an easy way to accelerate functionality in one's application. Wouldn't it be great if there were libraries that took advantage of GPUs? Luckily, NVIDIA provides a plethora of [GPU-accelerated libraries](https://developer.nvidia.com/gpu-accelerated-libraries), for linear algebra, signal processing, image and video processing. In this example, we'll focus on [cuBLAS](https://developer.nvidia.com/cublas), which is the GPU-accelerated version of BLAS.

Start by studying [cublas.cpp](/edit/cublas.cpp) containing the code. If you were to compare the *blas*() function from [blas.cpp](/edit/blas.cpp) to the *cublas*() function, you will notice there is a lot more going on in *cublas*(). For the time being, lets ignore everything but line 76. This is the cuBLAS GEMM call and it looks nearly identical to the regular BLAS call! This is deliberate and enables programmers to easily update code to take advantage of the GPU hardware.

Now let us take a look at everything else in the *cublas*() function. Like most things in life, there is no such thing as a free lunch. With GPU-accelerated libraries, programmers have to do more work to get the data 'ready'. In other words, data has to be sent to the GPU, if it's not already there, before work can be done on it. The result will then need to be transferred back to the CPU. In these example, matrices are prepopulated by the CPU and transferred to device memory on the GPU. This additional code is hidden from programmers when using OpenACC, but its execution time is not nearly as fast. This is a trade-off between implementation effort and performance.

Lastly, like OpenACC, cuBLAS doesn't use system environment variables. 

Please visit [NVIDIA DLI Education](https://www.nvidia.com/en-us/deep-learning-ai/education/) for an in-depth tutorial on cuBLAS and other GPU-accelerated libraries!

Implement the following suggestions to better understanding how they can affect the compile and/or runtime performance.

1. Change the optimization flag between `-O0`, `-O1`, `-O2`, and `-O3`.
2. Set the matrix to something smaller and larger than 1024.

In [ ]:
!nvcc -O2 -lcublas cublas.cpp -o cublas

In [ ]:
!./cublas 1024

## CUDA
Finally, what do you do if you compiler directives are not a good fit and GPU-accelerated libraries aren't available for your code base? That's were the [CUDA](https://developer.nvidia.com/how-to-cuda-c-cpp) API comes into play! It is the foundation of OpenACC and GPU-accelerated libraries. Programmers can use it to write code to fit whatever needs that have.

Start by studying [cuda.cu](/edit/cuda.cu) containing the code. Right off the bat, we see that the file has a ****.cu*** extension. This signifies to NVCC that the file we are trying to compile contains CUDA code. In the code, you will two additional functions related to cuda. At the moment, lets only focus on the *cuda*() function starting at line 70. Notice that similar steps are required to get the data ready as we saw when using cuBLAS. But instead of calling a library, we call a custom written *kernel* called *MatMulKernel*(). A kernel is a function that executes on a GPU and provides the ability to access additional features of the hardware and software. When we use custom kernels we also have to tell the compiler how many GPU threads we need. Threads are grouped into blocks which are then grouped into a grid. To learn more about the thread hiearchy, please read the [CUDA Programming Guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#programming-model). The kernel is then distributed to all the requested GPU threads to be executed independently.

Please visit [NVIDIA DLI Education](https://www.nvidia.com/en-us/deep-learning-ai/education/) for an in-depth tutorial on CUDA!

Implement the following suggestions to better understanding how they can affect the compile and/or runtime performance.

1. Change the optimization flag between `-O0`, `-O1`, `-O2`, and `-O3`.
2. Set the matrix to something smaller and larger than 1024.
3. Change the number of `BLOCK_SIZE` at line 21 to 8, 16, and 32.

In [ ]:
!nvcc -O2 cuda.cu -o cuda

In [ ]:
!./cuda 1024

## Conclusion
We hope you enjoyed this overview of CUDA and its accompanying resources. The next step to sign up for a [NVIDIA Developer](https://developer.nvidia.com/developer-program) account! And don't forget to check out the [NVIDIA Education](https://www.nvidia.com/en-us/deep-learning-ai/education/) section for more detailed training and resources.